# Toeplitz Convolution in 1D and 2D: Padding, Shifts, and Parameter Counting

In this notebook, we:
1. Demonstrate 1D convolution with a **Toeplitz** matrix.
2. Define and test **equivariance** over integer shifts.
3. Show how **padding** changes the output size and affects equivariance.
4. Move to a **2D** case (4×5 input), do an **`im2col`** explanation, and replicate the convolution via an explicitly filled `Linear` layer.
5. Wrap up with an exercise on **parameter counts**:
   - Full linear dimension,
   - Nonzero Toeplitz entries,
   - Actual number of kernel parameters.

We also set `torch.set_grad_enabled(False)` to ensure no gradient overhead while we do these manual operations.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

torch.set_grad_enabled(False)
print("torch.set_grad_enabled(False) is now active.")

## Part 1: 1D Convolution (Length=10)

We’ll do a **valid** 1D convolution:
- Input length = **10** (mostly zeros at edges). 
- A 3-tap kernel (length=3). 
- **No bias** in the convolution.

### 1.1 Creating and Filling the Conv1d Kernel
Let’s define a small kernel, e.g. \([1,2,1]\).

In [ ]:
# Our 1D input: shape (batch=1, channels=1, length=10)
x_1d = torch.tensor([[[0.,1.,2.,3.,4.,5.,6.,0.,0.,0.]]])
print("x_1d shape:", x_1d.shape)
print("x_1d:", x_1d)

# Conv1d with kernel_size=3, out_channels=1, no bias.
conv1d_layer = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=3, bias=False)

# Manually set the kernel to [1,2,1]
with torch.no_grad():
    # We'll do something like conv1d_layer.weight[0,0] = torch.tensor([1.,2.,1.])
    conv1d_layer.weight[0,0] = torch.tensor([1., 2., 1.])

print("Conv1D kernel:", conv1d_layer.weight)

# Forward pass (valid convolution -> output length = 10-3+1 = 8)
y_1d_conv = conv1d_layer(x_1d)
print("Output shape:", y_1d_conv.shape)
print("Output:", y_1d_conv)

$$\text{Output length is } 10 - 3 + 1 = 8.$$

### 1.2 Equivariance

A function \(f\) is **equivariant** to a transformation \(\tau\) if:
$$
f(\tau(x)) \;=\; \tau\bigl(f(x)\bigr).
$$
In the case of **shift** in 1D, \(\tau\) might shift the signal by \(n\) steps. Convolution with “valid” boundaries is only partially shift-equivariant: if you shift too far, the kernel will have no data to convolve. Let’s define a quick `shift(n)` function and test.


In [ ]:
def shift_1d(x, n):
    """
    Shift the 1D tensor by n steps.
    We'll keep the same length, so we lose data at one boundary
    and add zeros at the other.
    x: shape (batch=1, channel=1, length=L)
    n>0 => shift right, n<0 => shift left.
    """
    L = x.shape[-1]
    shifted = torch.zeros_like(x)

    if n >= 0:
        # shift right by n.
        # x[..., 0 : L-n] -> shifted[..., n : L]
        src_slice = x[..., 0 : L-n]
        dst_slice = shifted[..., n : L]
        dst_slice[:] = src_slice
    else:
        # shift left by |n|
        n_abs = -n
        src_slice = x[..., n_abs : L]
        dst_slice = shifted[..., 0 : L-n_abs]
        dst_slice[:] = src_slice

    return shifted

# Quick test of shift_1d:
print("Original x_1d:", x_1d)
x_1d_shifted2 = shift_1d(x_1d, 2)
print("Shifted by 2:", x_1d_shifted2)


Now, let's define a **range** of shifts (say \(-5\) to \(+5\)) and see if the convolution output is also shifted accordingly.
But because we do **valid** convolution, we expect that large shifts will break the alignment.

### 1.2.1 **Student Exercise**
Fill in the code below to:
1. Create `x_shifted = shift_1d(x_1d, n)`.
2. Compute `y_shifted = conv1d_layer(x_shifted)`.
3. Compare it to shifting the original `y_1d_conv` by `n` steps.
4. Observe for which `n` you get `y_shifted ≈ shift_1d(y_1d_conv, n)`.


In [ ]:
for n in range(-5, 6):
    x_shifted = shift_1d(x_1d, n)
    y_shifted = conv1d_layer(x_shifted)
    # We'll shift the original y_1d_conv by n as well, but that is shape (1,1,8)
    y_1d_shifted_ref = shift_1d(y_1d_conv, n)

    # Compare numerically (just print or store difference). We only do a small snippet.
    diff = (y_shifted - y_1d_shifted_ref).abs().sum()
    print(f"Shift n={n:2d}, difference={diff.item():.3f}")

print("\nWhere is it roughly zero? Discuss your observations.")

We expect small shifts (like \(-1,0,1\)) might show partial equivariance, but once the shift is big enough that the kernel can't align with the data region, it breaks.

---
### 1.3 Building an Explicit Toeplitz Matrix in `nn.Linear`

The output length is 8, and the input length is 10. So a fully connected layer from **10** to **8** can replicate the same transform if we fill it with the Toeplitz pattern:
$$
\text{weight} \in \mathbb{R}^{8 \times 10}, \quad y = \text{weight} \cdot x.
$$
We only need **3** unique weights \((w_0, w_1, w_2)\) repeated across the rows. 

**Student Exercise**: Fill in the matrix so that row 0 picks up \([x_0, x_1, x_2]\) with \([w_0, w_1, w_2]\), row 1 picks up \([x_1, x_2, x_3]\), etc.


In [ ]:
# Create the linear layer from 10->8, no bias.
fc1d = nn.Linear(in_features=10, out_features=8, bias=False)

# We'll manually fill it.
with torch.no_grad():
    fc1d.weight.zero_()

# Let's see the shape:
print("fc1d.weight.shape =", fc1d.weight.shape)
print("Initially:", fc1d.weight)

# STUDENT: fill the Toeplitz pattern. We know w=[1,2,1].
# row i => columns i..i+2 get [1,2,1]. We'll do a simple loop.
with torch.no_grad():
    wvals = [1., 2., 1.]
    for i in range(8):  # rows
        fc1d.weight[i, i  ] = wvals[0]
        fc1d.weight[i, i+1] = wvals[1]
        fc1d.weight[i, i+2] = wvals[2]

print("\nAfter filling:")
print(fc1d.weight)

# Compare with conv1d_layer results:
x_1d_flat = x_1d.view(1,10)  # shape (1,10)
y_fc1d = fc1d(x_1d_flat)
print("\nfc1d output shape:", y_fc1d.shape)
print("fc1d output:", y_fc1d)

diff_to_conv = y_fc1d.view(-1) - y_1d_conv.view(-1)
print("\nDifference vs conv:", diff_to_conv)


If all goes well, the difference should be near 0.

---
## Part 2: Adding Padding=1 in 1D

Now let's see how **padding** affects the output shape and equivariance. If we do `padding=1` with a `Conv1d`, a kernel of length=3, and input length=10, the output length remains **10** (since the kernel can start at the left edge with a pad). We’ll see that large shifts might cause boundary issues.

### 2.1 Padded Conv1d


In [ ]:
# We'll re-create a new conv1d for the padded case.
conv1d_pad = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=3, padding=1, bias=False)
with torch.no_grad():
    conv1d_pad.weight[0,0] = torch.tensor([1.,2.,1.])

# This should produce an output of length=10.
y_pad = conv1d_pad(x_1d)
print("Padding=1 -> output shape:", y_pad.shape)
print("Output:", y_pad)


### 2.2 Toeplitz Matrix for the Padded Case

Now the output is length=10, and the input is length=10, so we can do a `Linear(10,10, bias=False)` and fill it with the Toeplitz pattern for a 3-tap kernel **plus** the notion of 1 padding on each side. But effectively, the kernel can index beyond the left or right edges if we didn't pad. We'll represent that as extra columns if you wish. However, in many references, the Toeplitz approach for padding is effectively a 10×10 with some of the top-left corners hosting the kernel partial.

**Student**: Fill in the 10×10 so that it matches `padding=1` semantics. Or see below for the direct approach.


In [ ]:
fc1d_pad = nn.Linear(10, 10, bias=False)
with torch.no_grad():
    fc1d_pad.weight.zero_()

print("Initial fc1d_pad weights:", fc1d_pad.weight)

# We'll define a small function to fill in the padded topelitz.
def fill_toeplitz_padded(linear_layer, wvals):
    # length=10 in/out, kernel=3, padding=1 => each output i sees x[i-1], x[i], x[i+1], ignoring out-of-bounds.
    with torch.no_grad():
        linear_layer.weight.zero_()
        for out_i in range(10):
            # out_i indexes the output.
            # The kernel is [w0, w1, w2], which correspond to x[out_i-1], x[out_i], x[out_i+1].
            for k in range(3):
                # position in the input = out_i + (k-1)
                in_i = out_i + (k-1)
                if in_i>=0 and in_i<10:
                    linear_layer.weight[out_i, in_i] = wvals[k]

fill_toeplitz_padded(fc1d_pad, [1.,2.,1.])
print("\nAfter filling:")
print(fc1d_pad.weight)

# Compare with conv1d_pad
y_fc1d_pad = fc1d_pad(x_1d.view(1,10))
diff_pad = y_fc1d_pad.view(-1) - y_pad.view(-1)
print("\nDifference:", diff_pad)


If the difference is near zero, we’ve matched the padded convolution.

### 2.3 Check Equivariance with Padding=1
We can again try shifting the input in the range \(-5..5\). With padding, the output is always length=10. But does that make it fully shift-equivariant?

In practice, the *artificial* zeros at the edges might still cause boundary effects that break perfect equivariance once the shift is large enough.

> **Student**: Try `shift_1d(x_1d, n)`, pass to `conv1d_pad`, compare to shifting the *output* by `n`. Where does it break?

---
## Part 3: A 2D Example (4×5 input) and `im2col`

We shrink the 2D input to shape 4×5. A 3×3 kernel with **valid** convolution yields output shape 2×3.

### 3.1 The 2D Input


In [ ]:
img_2d = torch.tensor([
    [1,1,0,1,0],
    [0,1,1,1,1],
    [1,0,0,1,0],
    [1,1,1,0,0]
], dtype=torch.float)

print("Image shape: 4x5")
print(img_2d)
plt.figure(figsize=(4,3))
plt.imshow(img_2d, cmap='gray')
plt.title("Binary Image (4x5)")
plt.colorbar()
plt.show()

x_2d = img_2d.unsqueeze(0).unsqueeze(0)  # (1,1,4,5)
print("x_2d shape:", x_2d.shape)

### 3.2 Define a 3×3 Kernel and Convolve
The valid output shape is (4-3+1)×(5-3+1) = 2×3.


In [ ]:
conv2d = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, bias=False)
with torch.no_grad():
    # Let's define some pattern for the 3x3 kernel:
    # e.g.
    #  [ 1, 0, 1]
    #  [-1, 2, 0]
    #  [ 1, 1,-1]
    conv2d.weight[0,0] = torch.tensor([
        [ 1.,  0.,  1.],
        [-1.,  2.,  0.],
        [ 1.,  1., -1.]
    ])

y_2d = conv2d(x_2d)
print("2D conv output shape:", y_2d.shape)  # (1,1,2,3)
print("Output:", y_2d)

plt.figure(figsize=(3,2))
plt.imshow(y_2d[0,0,:,:], cmap='viridis')
plt.title("Conv2d Output (2x3)")
plt.colorbar()
plt.show()

### 3.3 `im2col` and Flattening Explanation

If we flatten the **4×5** input in **row-major** order, we get 20 entries:
$$
\text{vec}(X) = [\,x_{0,0},\; x_{0,1},\;\dots,\; x_{0,4},\; x_{1,0},\dots, x_{3,4}\,]^\top \in \mathbb{R}^{20}.
$$
And the **2×3** output is 6 entries:
$$
\text{vec}(Y) = [\,y_{0,0},\; y_{0,1},\; y_{0,2},\; y_{1,0},\; y_{1,1},\; y_{1,2}\,]^\top \in \mathbb{R}^{6}.
$$
The 3×3 kernel has 9 parameters, repeated in a **block-Toeplitz** pattern of size (6×20).

**`im2col`** would produce a (9 × 6) matrix of patches. Each patch is 3×3 flattened to 9, stacked across 2×3 = 6 positions. Multiplying that by the 9 kernel parameters yields the same result.

### 3.4 Exercise: Fill a `Linear(20,6)` with the Toeplitz Layout
We create a `nn.Linear(20, 6, bias=False)`. Then for each output position `(r,c)`, we place the kernel into the correct 9 positions referencing `(r+dr, c+dc)` in the input. Students fill in the code.


In [ ]:
fc2d = nn.Linear(in_features=20, out_features=6, bias=False)
with torch.no_grad():
    fc2d.weight.zero_()

print("fc2d weight shape:", fc2d.weight.shape)  # (6,20)
print("Initially:", fc2d.weight)

def fill_block_toeplitz_2d(linear_layer, kernel_3x3):
    # kernel shape: (3,3)
    # output shape: (2,3) => 6 positions
    # input shape:  (4,5) => 20 positions in flatten
    with torch.no_grad():
        linear_layer.weight.zero_()
        out_index = 0
        for r_out in range(2):
            for c_out in range(3):
                out_index = r_out*3 + c_out
                # For each kernel cell
                for kr in range(3):
                    for kc in range(3):
                        val = kernel_3x3[kr,kc].item()
                        r_in = r_out + kr
                        c_in = c_out + kc
                        in_index = r_in*5 + c_in  # flatten row-major
                        linear_layer.weight[out_index, in_index] = val

# Fill it:
fill_block_toeplitz_2d(fc2d, conv2d.weight[0,0])
print("After fill:", fc2d.weight)

# Multiply x_2d flattened
x_2d_flat = x_2d.view(1,20)
y_fc2d = fc2d(x_2d_flat)  # shape (1,6)
y_fc2d_reshaped = y_fc2d.view(2,3)

print("fc2d output (2x3):\n", y_fc2d_reshaped)
diff_2d = y_fc2d_reshaped - y_2d[0,0]
print("Difference vs conv2d:", diff_2d)

# Visual comparison
plt.figure(figsize=(8,3))
plt.subplot(1,2,1)
plt.imshow(y_2d[0,0].detach(), cmap='viridis')
plt.title("Conv2D Output")
plt.colorbar()

plt.subplot(1,2,2)
plt.imshow(y_fc2d_reshaped.detach(), cmap='viridis')
plt.title("Linear Block-Toeplitz Output")
plt.colorbar()
plt.show()

If the difference is near zero, we have successfully replicated the 2D convolution with a single large matrix that repeats the 9 kernel parameters in a block-Toeplitz pattern.

---
## Part 4: Parameter Counts

Finally, let’s highlight the massive difference in parameter counts:
1. **Full Linear**: (out_features) × (in_features). E.g., 6×20=120 in the 2D case.
2. **Block-Toeplitz**: Many repeated entries. Potentially the same matrix shape, but large swaths are repeated. The actual number of **non-zero** unique positions can be computed (9 repeated in different rows for each output patch, i.e. 54 total nonzero, but only 9 unique parameter values if we allow full weight‐sharing). 
3. **Convolution Kernel**: 3×3=9 parameters.

### Student Exercise
1. In the 1D case (length=10, output=8), the full linear is 8×10=80. The Toeplitz has 8 rows with 3 nonzero entries each → 24 nonzero. But only 3 actual kernel parameters. 
2. In the 2D case (20→6), the full linear is 120. The block-Toeplitz might have 6 patches × 9 = 54 nonzero, but only 9 distinct kernel values. 

Confirm these numbers or fill them in for your assignment.

---
## Conclusion

We've now:
- Demonstrated **Toeplitz** and **block-Toeplitz** construction for valid convolution in 1D and 2D.
- Seen how **padding** changes the dimension and partial equivariance.
- Understood that **im2col** is effectively building the block-Toeplitz or patch-based multiplication.
- Highlighted the difference between a fully dense linear map and a convolution with **weight sharing**.

Feel free to explore different kernels, shift amounts, or pad sizes to deepen understanding.

## End of Notebook